In [2]:
# test pyspark type

sc.master[0:5]

u'local'

In [3]:
# choose data source

global Path
if sc.master[0:5]=="local":
    Path = "file:/home/cloudera/als/"
else:
    Path="/user/cloudera/als/"

In [4]:
# loading csv & count all rdd

rawUserData = sc.textFile(Path + "ratings.csv")
rawUserData.count()

24404097

In [5]:
# see data

rawUserData.take(10)

[u'userId,movieId,rating,timestamp',
 u'1,122,2.0,945544824',
 u'1,172,1.0,945544871',
 u'1,1221,5.0,945544788',
 u'1,1441,4.0,945544871',
 u'1,1609,3.0,945544824',
 u'1,1961,3.0,945544871',
 u'1,1972,1.0,945544871',
 u'2,441,2.0,1008942733',
 u'2,494,2.0,1008942733']

In [6]:
# skip first row

header = rawUserData.first()
rawUserData = rawUserData.filter(lambda line:line != header) 
rawUserData.take(10)

[u'1,122,2.0,945544824',
 u'1,172,1.0,945544871',
 u'1,1221,5.0,945544788',
 u'1,1441,4.0,945544871',
 u'1,1609,3.0,945544824',
 u'1,1961,3.0,945544871',
 u'1,1972,1.0,945544871',
 u'2,441,2.0,1008942733',
 u'2,494,2.0,1008942733',
 u'2,1193,4.0,1008942667']

In [7]:
# import Rating model

from pyspark.mllib.recommendation import Rating

In [8]:
# choose 1-3 colume to Operation (list type)

rawRatings = rawUserData.map(lambda line:line.split(",")[:4])
rawRatings.take(10)

[[u'1', u'122', u'2.0', u'945544824'],
 [u'1', u'172', u'1.0', u'945544871'],
 [u'1', u'1221', u'5.0', u'945544788'],
 [u'1', u'1441', u'4.0', u'945544871'],
 [u'1', u'1609', u'3.0', u'945544824'],
 [u'1', u'1961', u'3.0', u'945544871'],
 [u'1', u'1972', u'1.0', u'945544871'],
 [u'2', u'441', u'2.0', u'1008942733'],
 [u'2', u'494', u'2.0', u'1008942733'],
 [u'2', u'1193', u'4.0', u'1008942667']]

In [9]:
type(rawRatings)

pyspark.rdd.PipelinedRDD

In [10]:
# establish ratingsRDD (tuple type)

RatingsRDD = rawRatings.map(lambda x:(x[0],x[1],x[2]))
RatingsRDD.take(10)

[(u'1', u'122', u'2.0'),
 (u'1', u'172', u'1.0'),
 (u'1', u'1221', u'5.0'),
 (u'1', u'1441', u'4.0'),
 (u'1', u'1609', u'3.0'),
 (u'1', u'1961', u'3.0'),
 (u'1', u'1972', u'1.0'),
 (u'2', u'441', u'2.0'),
 (u'2', u'494', u'2.0'),
 (u'2', u'1193', u'4.0')]

In [11]:
type(RatingsRDD)

pyspark.rdd.PipelinedRDD

In [12]:
# count all rdd

numRatings = RatingsRDD.count()
numRatings

24404096

In [13]:
# count unique user

numUsers = RatingsRDD.map(lambda x:x[0]).distinct().count()
numUsers

259137

In [14]:
# count unique movie

numMovies = RatingsRDD.map(lambda x:x[1]).distinct().count()
numMovies

39443

In [15]:
# import ALS model

from pyspark.mllib.recommendation import ALS


In [16]:
# import os
# os.environ["PYSPARK_PYTHON"] = os.environ["CONDA_DIR"] + "/envs/python2/bin/python2"

In [17]:
# Explicit rating
# ALS.train(ratings, rank, iteration=5(default), lambda_=0.01(default))
# return "MatrixFactorizationModel"

# import numpy

model = ALS.train(RatingsRDD, 10, 10, 0.01)
print model

In [19]:
# Implicit rating
# ALS.trainImplicit(ratings, rank, iteration=5(default), lambda_=0.01(default))
# return "MatrixFactorizationModel"


In [20]:
type(model)

pyspark.mllib.recommendation.MatrixFactorizationModel

In [21]:
# Recommend Movies
# model.recommendProducts(user_id, recommend_movie_num)

model.recommendProducts(100,5)

[Rating(user=100, product=142094, rating=8.42171155371899),
 Rating(user=100, product=153010, rating=8.225754078347764),
 Rating(user=100, product=153184, rating=8.225754078347764),
 Rating(user=100, product=153014, rating=7.403178598082263),
 Rating(user=100, product=153018, rating=7.403178598082263)]

In [23]:
# Recommend Movie Points
# model.predict(user_id, recommend_movie_id)

model.predict(100,1141)

3.2500968010532523

In [55]:
# Recommend Users
# model.recommendUsers(product_id, recommend_user_num)

model.recommendUsers(product = 200, num = 5)

[Rating(user=90323, product=200, rating=7.140733394855737),
 Rating(user=164593, product=200, rating=6.915808259127213),
 Rating(user=210231, product=200, rating=6.698500480120079),
 Rating(user=11038, product=200, rating=6.692790117700989),
 Rating(user=234323, product=200, rating=6.380934034635413)]

In [177]:
# show movie name

itemRDD = sc.textFile(Path+"movies.csv")
itemRDD.take(5)
# itemRDD.count()

[u'movieId,title,genres',
 u'1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 u'2,Jumanji (1995),Adventure|Children|Fantasy',
 u'3,Grumpier Old Men (1995),Comedy|Romance',
 u'4,Waiting to Exhale (1995),Comedy|Drama|Romance']

In [178]:
# skip first row

header = itemRDD.first()
itemRDD = itemRDD.filter(lambda line: line != header)
           
# type(itemRDD)    
itemRDD.take(5)
# itemRDD.count()

[u'1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 u'2,Jumanji (1995),Adventure|Children|Fantasy',
 u'3,Grumpier Old Men (1995),Comedy|Romance',
 u'4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 u'5,Father of the Bride Part II (1995),Comedy']

In [195]:
# dict with movie id & movie name

movieTitle = itemRDD.map(lambda line: line.split(",")) \
                    .map(lambda a: (float(a[0]),a[1])) \
                    .collectAsMap()

        
# <note>  u''  is str       
# movieTitle = itemRDD.map(lambda line: line.split(",")) \
#                     .map(lambda a: (a[0],a[1])) \
#                     .collectAsMap()

        
# movieTitle
type(movieTitle)
# len(movieTitle)

# movieTitle[150000]
# movieTitle[6]

dict

In [189]:
# type list

movieTitle.items()[ :6]
# movieTitle.items()
# type(movieTitle.items())


[(131072.0, u'Jesus liebt mich (2012)'),
 (1.0, u'Toy Story (1995)'),
 (2.0, u'Jumanji (1995)'),
 (3.0, u'Grumpier Old Men (1995)'),
 (4.0, u'Waiting to Exhale (1995)'),
 (5.0, u'Father of the Bride Part II (1995)')]

In [199]:
recommendP = model.recommendProducts(100,5)

for row in recommendP:
#     print row
    print "target user: " + str(row[0]) + "    " + \
            "recommend movie: "+ str(movieTitle[row[1]]) + "    " + \
            "point: " + str(row[2])
            
# type(recommendP)

target user: 100    recommend movie: Alice in Wonderland (1915)    point: 8.42171155372
target user: 100    recommend movie: Vergeef me    point: 8.22575407835
target user: 100    recommend movie: L'ange (1983)    point: 8.22575407835
target user: 100    recommend movie: The Bed You Sleep In (1993)    point: 7.40317859808
target user: 100    recommend movie: Nightfall (1999)    point: 7.40317859808
